# Decentmon data preparation

In [1]:
import pandas as pd

from decmon.constants import *
from decmon.df_utils import *
from decmon.filter import *
from decmon.plotter import *


In [2]:
SUBDIR = 3

In [3]:
path = f"{INPUT_DIR}/{SUBDIR}"

f_df = [load_simulation_data(f"{path}/{i+1}.log") for i in PATTERNS]
f_df[0].head()

,formula_id,x,cent_trace_len,decent_trace_len,odecent_trace_len,cent_num_mess,decent_num_mess,odecent_num_mess,cent_size_mess,decent_size_mess,odecent_size_mess,cent_nb_progressions,decent_nb_progressions,odecent_nb_progressions,formula,trace
0,3073,1,3,5,4,6,7,3,1.0,3.74,1.77,12,53,17,"Until (Var ""c"", Var ""a"")",{ |b|c} ; { |b|c} ; { |b| } ; { | | } ; {a| |...
1,3073,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""c"")",{ | | } ; {a|b|c} ; { |b| } ; {a|b| } ; {a| |...
2,3073,1,2,3,3,3,3,2,1.0,3.50,1.66,8,17,12,"Until (Var ""a"", Var ""c"")",{a| | } ; { |b|c} ; {a|b|c} ; { | | } ; {a|b|...
3,3073,1,2,3,2,3,2,0,1.0,1.66,0.00,4,8,4,"Glob (Var ""c"")",{ |b|c} ; {a|b| } ; {a| |c} ; {a| |c} ; {a| |...
4,3073,1,2,3,2,3,2,0,1.0,1.66,0.00,4,8,4,"Glob (Var ""a"")",{a| | } ; { | |c} ; {a|b|c} ; { |b| } ; { |b|...


In [4]:
strategies_f_df = [split_by_dictionary(f_df[i], STRATEGIES) for i in PATTERNS]

print(f"Datasets: {len(strategies_f_df)}. Strategies: {len(strategies_f_df[0])}. Single dataset Formulae:{len(strategies_f_df[0][0])}")
sep_strat_f_df = [pd.concat(strategies_f_df[i], ignore_index=True) for i in PATTERNS]
sep_strat_f_df[0].head()

Datasets: 8. Strategies: 3. Single dataset Formulae:3073


,formula_id,x,trace_len,num_mess,size_mess,nb_progressions,formula,trace,strategy
0,3073,1,3,6,1.0,12,"Until (Var ""c"", Var ""a"")",{ |b|c} ; { |b|c} ; { |b| } ; { | | } ; {a| |...,orchestration
1,3073,1,2,3,1.0,3,"Next (Var ""c"")",{ | | } ; {a|b|c} ; { |b| } ; {a|b| } ; {a| |...,orchestration
2,3073,1,2,3,1.0,8,"Until (Var ""a"", Var ""c"")",{a| | } ; { |b|c} ; {a|b|c} ; { | | } ; {a|b|...,orchestration
3,3073,1,2,3,1.0,4,"Glob (Var ""c"")",{ |b|c} ; {a|b| } ; {a| |c} ; {a| |c} ; {a| |...,orchestration
4,3073,1,2,3,1.0,4,"Glob (Var ""a"")",{a| | } ; { | |c} ; {a|b|c} ; { |b| } ; { |b|...,orchestration


In [5]:
strategies_f_df

[[      formula_id  x  trace_len  num_mess  size_mess  nb_progressions   
  0           3073  1          3         6        1.0               12  \
  1           3073  1          2         3        1.0                3   
  2           3073  1          2         3        1.0                8   
  3           3073  1          2         3        1.0                4   
  4           3073  1          2         3        1.0                4   
  ...          ... ..        ...       ...        ...              ...   
  3068        3073  4         11        30        1.0              125   
  3069        3073  4         11        30        1.0               62   
  3070        3073  4         17        48        1.0              184   
  3071        3073  4         13        36        1.0              141   
  3072        3073  4         15        42        1.0              144   
  
                                                  formula   
  0                             Until (Var "c", 

In [6]:
merged_metrics_df = [extract_metrics(sep_strat_f_df[i]) for i in PATTERNS]
merged_metrics_df[0].max()

formula_id             3073
strategy      orchestration
value                5120.0
metric            trace_len
dtype: object

## Formula Encoding

In [7]:
def encode_normalize(df):
    return extract_ops(normalize_ops(df), encoding='urgency')


df_f_ops = [encode_normalize(sep_strat_f_df[i]) for i in PATTERNS]

In [8]:
df_f_ops[0].head()

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,2,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,0,1.0,-1.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,0,1.0,-1.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
plot_atoms_variation(df_f_ops, SUBDIR)

## Plotting

In [ ]:
# for i in PATTERNS:
#     grid_plot = plot_grid_barplots(merged_metrics_df[i], grid_cell_field= 'metric',
#               y_axis = 'value', x_axis = 'strategy',
#               order = ['orchestration', 'migration', 'choreography'])

#grid_plot.savefig("output/out.png")

In [ ]:
merged_metrics_dfp = prepare_with_pattern(merged_metrics_df)
plot = plot_metrics_by_patterns(merged_metrics_dfp)
plot.savefig(f"output/pattern_{SUBDIR}.png")
plot

In [ ]:
expanded_f_df = [sep_strat_f_df[i].merge(df_f_ops[i], left_index=True, right_index=True) for i in PATTERNS]
expanded_f_df[0].head()

In [ ]:
for i in PATTERNS:
    expanded_f_df[i].strategy = pd.Categorical(expanded_f_df[i].strategy)
    expanded_f_df[i]['strategy_id'] = expanded_f_df[i].strategy.cat.codes

full_scale_df_3 = pd.concat(expanded_f_df)
full_scale_df_3

In [ ]:
full_scale_df_3[200].unique()

In [ ]:
# TODO:
# -- pycaret for classification
# Action points
# - higher number of nodes (5, 7, 9?)
# - statistics about the learning
# - test other algorithms (pycaret)
